### 1. Replication of Paper (Linear 2SLS)

In [60]:
import pandas as pd
import numpy as np
from statsmodels.api import OLS, add_constant

# =========================================================
# 0. LOAD DATA
# =========================================================
df = pd.read_csv("/Users/choi/Desktop/1118_assignment3/data/projectSpecific/panel_forDML.csv")
df = df.sort_values(["muni_code", "year"]).reset_index(drop=True)

# =========================================================
# 1. LAGS (Stata L.)
# =========================================================
group = df.groupby("muni_code")

df["L_fines_count"]   = group["fines_count"].shift(1)
df["L_deter_cloud"]   = group["deter_cloud"].shift(1)
df["L_weather_rain"]  = group["weather_rain"].shift(1)
df["L_weather_temp"]  = group["weather_temp"].shift(1)

price_vars = [
    "priceWgtNdx_brzl_s1_cattle", "priceWgtNdx_brzl_s1_sugar",
    "priceWgtNdx_brzl_s1_corn",   "priceWgtNdx_brzl_s1_soybean",
    "priceWgtNdx_brzl_s1_cassava","priceWgtNdx_brzl_s1_rice"
]

for v in price_vars:
    df[f"L_{v}"] = group[v].shift(1)
    df[f"L_{v.replace('s1','s2')}"] = group[v.replace("s1","s2")].shift(1)

# =========================================================
# 2. SAMPLE SELECTION — FILTER FIRST (CRUCIAL)
# =========================================================
df = df[(df["year"] >= 2007) & (df["year"] <= 2016)]
df = df.dropna(subset=["L_fines_count", "L_deter_cloud"])
df = df.reset_index(drop=True)

# =========================================================
# 3. WITHIN TRANSFORMATION
# =========================================================
def within(df, group, cols):
    return df[cols] - df.groupby(group)[cols].transform("mean")

y_var = "prodes_deforest_ihs"
# y_var = "prodes_deforest_log"
# y_var = "prodes_deforest_dma_100"
# y_var = "prodes_deforest_dbm"

endog = "L_fines_count"
instr = "L_deter_cloud"

controls = ["L_weather_rain", "L_weather_temp", "prodes_nonobs", "prodes_cloud"]
for v in price_vars:
    controls += [v, f"L_{v}", f"L_{v.replace('s1','s2')}"]

demean_vars = [y_var, endog, instr] + controls
df_w = within(df, "muni_code", demean_vars)

# =========================================================
# 4. YEAR FE
# =========================================================
year_fe = pd.get_dummies(df["year"], prefix="year", drop_first=True)
X_controls = pd.concat([df_w[controls], year_fe], axis=1)

# =========================================================
# 5. FIRST STAGE
# =========================================================
Z = df_w[[instr]]
X1 = pd.concat([Z, X_controls], axis=1)
X1 = X1.copy()          # ensure no view
X1.insert(0, "const", 1.0)


X1 = X1.apply(pd.to_numeric, errors="coerce")
y1 = pd.to_numeric(df_w[endog], errors="coerce")

mask = X1.notnull().all(axis=1) & y1.notnull()
X1 = X1.loc[mask]
y1 = y1.loc[mask]

X1 = X1.astype(float)
y1 = y1.astype(float)


fs = OLS(y1, X1).fit()
print("\n=========================== FIRST STAGE ===========================")
print(fs.summary())
print("\nFirst-stage F-statistic:", fs.fvalue)

x_hat = fs.fittedvalues

# =========================================================
# 6. SECOND STAGE
# =========================================================
X2 = pd.concat([x_hat.rename("xhat"), X_controls.loc[mask]], axis=1)
X2 = X2.copy()
X2.insert(0, "const", 1.0)


y2 = df_w[y_var].loc[mask]

X2 = X2.astype(float)
y2 = y2.astype(float)


ss = OLS(y2, X2).fit()
# print("\n=========================== SECOND STAGE (NO CLUSTER) ===========================")
# print(ss.summary())

# =========================================================
# 7. TWO-WAY CLUSTERED SE
# =========================================================

# Convert cluster variable to numeric codes
df["cl_microYear_code"] = df["cl_microYear"].astype("category").cat.codes

clusters = df.loc[mask, ["muni_code", "cl_microYear_code"]]

ss_cluster = ss.get_robustcov_results(
    cov_type="cluster",
    groups=clusters,
)

print("\n=========================== SECOND STAGE (TWO-WAY CLUSTERED SE) ===========================")
print(ss_cluster.summary())



=========================== FIRST STAGE ===========================
                            OLS Regression Results                            
Dep. Variable:          L_fines_count   R-squared:                       0.031
Model:                            OLS   Adj. R-squared:                  0.025
Method:                 Least Squares   F-statistic:                     5.111
Date:                Mon, 01 Dec 2025   Prob (F-statistic):           3.26e-19
Time:                        13:25:49   Log-Likelihood:                -22346.
No. Observations:                5210   AIC:                         4.476e+04
Df Residuals:                    5177   BIC:                         4.498e+04
Df Model:                          32                                         
Covariance Type:            nonrobust                                         
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------

### 2. Double‐Selection LASSO 

In [87]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from statsmodels.api import OLS, add_constant

# =========================================================
# 0. LOAD AND PREPARE DATA
# =========================================================
df = pd.read_csv("/Users/choi/Desktop/1118_assignment3/data/projectSpecific/panel_forDML.csv")
df = df.sort_values(["muni_code", "year"]).reset_index(drop=True)

# lags
group = df.groupby("muni_code")
df["L_fines_count"]   = group["fines_count"].shift(1)
df["L_deter_cloud"]   = group["deter_cloud"].shift(1)
df["L_weather_rain"]  = group["weather_rain"].shift(1)
df["L_weather_temp"]  = group["weather_temp"].shift(1)

price_vars = [
    "priceWgtNdx_brzl_s1_cattle","priceWgtNdx_brzl_s1_sugar",
    "priceWgtNdx_brzl_s1_corn","priceWgtNdx_brzl_s1_soybean",
    "priceWgtNdx_brzl_s1_cassava","priceWgtNdx_brzl_s1_rice"
]
for v in price_vars:
    df[f"L_{v}"] = group[v].shift(1)
    df[f"L_{v.replace('s1','s2')}"] = group[v.replace("s1","s2")].shift(1)

# sample: 2007–2016
df = df[(df["year"] >= 2007) & (df["year"] <= 2016)]
df = df.dropna(subset=["L_fines_count","L_deter_cloud"]).reset_index(drop=True)

# =========================================================
# 1. MUNICIPALITY FE (WITHIN TRANSFORMATION)
# =========================================================
def within(df, group, cols):
    return df[cols] - df.groupby(group)[cols].transform("mean")

# y_var = "prodes_deforest_ihs"
# y_var = "prodes_deforest_log"
# y_var = "prodes_deforest_dma_100"
y_var = "prodes_deforest_dbm"

endog = "L_fines_count"        # y1 in your notes
instr = "L_deter_cloud"        # Z

controls = ["L_weather_rain","L_weather_temp","prodes_nonobs","prodes_cloud"]
for v in price_vars:
    controls += [v, f"L_{v}", f"L_{v.replace('s1','s2')}"]

demean_vars = [y_var, endog, instr] + controls
df_w = within(df, "muni_code", demean_vars)

# =========================================================
# 2. YEAR FIXED EFFECTS
# =========================================================
year_fe = pd.get_dummies(df["year"], prefix="year", drop_first=True)
year_fe = year_fe.astype(float)
X_controls = pd.concat([df_w[controls], year_fe], axis=1)

In [89]:
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import StandardScaler
from statsmodels.api import OLS, add_constant

# =========================================================
# 3. DOUBLE-SELECTION LASSO TO CHOOSE CONTROLS (LINEAR)
# =========================================================

# Outcome, endogenous, instrument (demeaned)
y_ds = df_w[y_var].values          # prodes_deforest_ihs (demeaned) 
D_ds = df_w[endog].values          # L_fines_count (demeaned) 
Z_ds = df_w[instr].values          # L_deter_cloud (demeaned) 

# Controls for LASSO: ONLY X, NO instrument
X_lasso = X_controls.copy()        # DataFrame of controls (demeaned + year FE)
control_names = X_lasso.columns

# Standardize controls for LASSO
scaler = StandardScaler()
X_std = scaler.fit_transform(X_lasso.values)

# ---- LASSO 1: deforest ~ controls ----
lasso_y = LassoCV(cv=5, random_state=123).fit(X_std, y_ds)
sel_y_idx = np.where(lasso_y.coef_ != 0)[0]

# ---- LASSO 2: fines_count ~ controls ----
lasso_D = LassoCV(cv=5, random_state=123).fit(X_std, D_ds)
sel_D_idx = np.where(lasso_D.coef_ != 0)[0]

# ---- Union of selected controls ----
sel_union_idx = np.unique(np.concatenate([sel_y_idx, sel_D_idx]))
selected_cols = control_names[sel_union_idx]

print("Selected controls by double LASSO:")
print(list(selected_cols))

X_sel = X_controls[selected_cols]      # DataFrame of selected controls (demeaned)

# =========================================================
# 4. LINEAR 2SLS WITH SELECTED CONTROLS
# =========================================================
print("\n========== FIRST STAGE (OLS): L_fines_count ~ L_deter_cloud + selected X ==========")

# First stage: L_fines_count ~ L_deter_cloud + selected_controls
X_first = pd.concat([
    df_w[[instr]],   # L_deter_cloud (demeaned instrument)
    X_sel            # selected controls
], axis=1)
X_first = add_constant(X_first)       # add intercept

first_stage = OLS(D_ds, X_first).fit()
print(first_stage.summary())

# Predicted fines (linear first stage)
fines_hat_ds = first_stage.fittedvalues

# print("\n========== SECOND STAGE (OLS): deforest ~ fines_hat_ds + selected X ==========")

# Second stage: prodes_deforest_ihs ~ fines_hat_ds + selected_controls
X_second = pd.concat([
    pd.Series(fines_hat_ds, name="fines_hat_ds"),
    X_sel
], axis=1)
X_second = add_constant(X_second)

second_stage = OLS(y_ds, X_second).fit()
# print(second_stage.summary())

# clustering
clusters = df.loc[X_second.index, ["muni_code", "cl_microYear"]].copy()
clusters["muni_code"] = clusters["muni_code"].astype(int)
clusters["cl_microYear"] = clusters["cl_microYear"].astype("category").cat.codes.astype(int)

ss_cluster = second_stage.get_robustcov_results(
    cov_type="cluster",
    groups=clusters
)

print("\n========== SECOND STAGE (Two-Way Clustered SE) ==========")
print(ss_cluster.summary())


Selected controls by double LASSO:
['L_weather_rain', 'L_weather_temp', 'prodes_nonobs', 'prodes_cloud', 'priceWgtNdx_brzl_s1_cattle', 'L_priceWgtNdx_brzl_s1_cattle', 'L_priceWgtNdx_brzl_s1_sugar', 'priceWgtNdx_brzl_s1_corn', 'L_priceWgtNdx_brzl_s1_corn', 'L_priceWgtNdx_brzl_s1_soybean', 'priceWgtNdx_brzl_s1_cassava', 'L_priceWgtNdx_brzl_s1_cassava', 'L_priceWgtNdx_brzl_s2_cassava', 'priceWgtNdx_brzl_s1_rice', 'L_priceWgtNdx_brzl_s1_rice', 'L_priceWgtNdx_brzl_s2_rice', 'year_2008', 'year_2009', 'year_2010', 'year_2011', 'year_2012', 'year_2013', 'year_2014', 'year_2015', 'year_2016']

========== FIRST STAGE (OLS): L_fines_count ~ L_deter_cloud + selected X ==========
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.030
Model:                            OLS   Adj. R-squared:                  0.025
Method:                 Least Squares   F-statistic:                     6.230
Date:   

### 3. Partially Linear DML-IV (RF)

In [134]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from statsmodels.api import OLS, add_constant

# =========================================================
# 3. PARTIALLY LINEAR DML (ML TO PARTIAL OUT CONTROLS)
# =========================================================

# variables (demeaned)
Y = df_w[y_var].values           # deforest
D = df_w[endog].values           # fines
Z = df_w[instr].values           # cloud
X = X_controls.values            # nonlinear controls (demeaned + year FE)

K = 5
kf = KFold(n_splits=K, shuffle=True, random_state=42)

# storage for cross-fitted residuals
Y_res = np.zeros_like(Y)
D_res = np.zeros_like(D)

# ML model for nonlinear controls
rf = RandomForestRegressor(
    n_estimators=500,
    min_samples_leaf=5,
    max_depth=None,
    random_state=42
)

for train_idx, test_idx in kf.split(X):

    # --- ML: predict E[Y|X] ---
    rf.fit(X[train_idx], Y[train_idx])
    Y_res[test_idx] = Y[test_idx] - rf.predict(X[test_idx])

    # --- ML: predict E[D|X] ---
    rf.fit(X[train_idx], D[train_idx])
    D_res[test_idx] = D[test_idx] - rf.predict(X[test_idx])


# =========================================================
# 4. DML-IV: LINEAR IV ON RESIDUALS
# =========================================================

# FIRST STAGE: D_res ~ Z_res
X_first = add_constant(Z)
first_stage = OLS(D_res, X_first).fit()
D_hat = first_stage.fittedvalues

print(first_stage.summary())

# SECOND STAGE: Y_res ~ D_hat
X_second = add_constant(D_hat)
second_stage = OLS(Y_res, X_second).fit()

# print(second_stage.summary())


# =========================================================
# 5. OLS Cluster SE (NOT ROBUST TO DML?)
# =========================================================

# If you want cluster-robust SEs, same code as before:
clusters = df[["muni_code","cl_microYear"]].copy()
clusters["muni_code"] = clusters["muni_code"].astype(int)
clusters["cl_microYear"] = clusters["cl_microYear"].astype("category").cat.codes.astype(int)

ss_cluster = second_stage.get_robustcov_results( 
    cov_type="cluster",
    groups=clusters
)
print("\n========== DML-IV with Clustered SE ==========")
print(ss_cluster.summary())


                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     1.347
Date:                Mon, 01 Dec 2025   Prob (F-statistic):              0.246
Time:                        20:42:31   Log-Likelihood:                -22376.
No. Observations:                5210   AIC:                         4.476e+04
Df Residuals:                    5208   BIC:                         4.477e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0783      0.246     -0.319      0.7

In [135]:
import numpy as np
from scipy.stats import norm

# from second-stage OLS on residuals
theta_hat = second_stage.params[1]
theta_hat

# ------------------------------------------------------------
# 0. Required objects
# ------------------------------------------------------------

# cluster IDs as 1D numpy arrays
cluster1 = np.asarray(clusters["muni_code"].values)
cluster2 = np.asarray(clusters["cl_microYear"].values)

# orthogonal scores: ψ_i = Z_res * (Y_res - theta_hat * D_res)
psi = Z * (Y_res - theta_hat * D_res)

# Jacobian:  J = -(1/n) * Σ Z_res * D_res
J = -np.mean(Z * D_res)

n = len(psi)

# ------------------------------------------------------------
# 1. Compute cluster sums of ψ_i
# ------------------------------------------------------------
def cluster_sum(psi_vec, cluster_id_vec):
    cluster_id_vec = np.asarray(cluster_id_vec)
    out = {}
    for cid in np.unique(cluster_id_vec):
        mask = (cluster_id_vec == cid)
        out[cid] = psi_vec[mask].sum()
    return out

# cluster 1 sums
C1 = cluster_sum(psi, cluster1)
# cluster 2 sums
C2 = cluster_sum(psi, cluster2)

# intersection clusters: create a single combined cluster label
cluster12 = np.array([f"{a}_{b}" for a, b in zip(cluster1, cluster2)])
C12 = cluster_sum(psi, cluster12)

# ------------------------------------------------------------
# 2. Variance components
# ------------------------------------------------------------
V1  = sum(v**2 for v in C1.values())
V2  = sum(v**2 for v in C2.values())
V12 = sum(v**2 for v in C12.values())

# Two-way cluster variance (Cameron–Gelbach–Miller)
cluster_var = (V1 + V2 - V12) / (n**2)

# ------------------------------------------------------------
# 3. Delta method for θ (Jacobian scaling)
# ------------------------------------------------------------
Var_theta = cluster_var / (J**2)
se_dml_tw = np.sqrt(Var_theta)

# ------------------------------------------------------------
# 4. CI
# ------------------------------------------------------------
z_95 = norm.ppf(0.975)
ci_low = theta_hat - z_95 * se_dml_tw
ci_high = theta_hat + z_95 * se_dml_tw

print(f"Theta_hat (DML-IV): {theta_hat:.6f}")
print(f"Two-Way Clustered DML SE: {se_dml_tw:.6f}")
print(f"95% CI (two-way cluster): ({ci_low:.6f}, {ci_high:.6f})")


Theta_hat (DML-IV): -0.054059
Two-Way Clustered DML SE: 0.075959
95% CI (two-way cluster): (-0.202935, 0.094817)


### 4. Partially Linear DML-IV (NN)

In [136]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import KFold
from statsmodels.api import OLS, add_constant
import numpy as np

# =========================================================
# 3. PARTIALLY LINEAR DML (ML TO PARTIAL OUT CONTROLS)
# =========================================================

# variables (demeaned)
Y = df_w[y_var].values           # deforest
D = df_w[endog].values           # fines
Z = df_w[instr].values           # cloud instrument
X = X_controls.values            # nonlinear controls

K = 5
kf = KFold(n_splits=K, shuffle=True, random_state=42)

# storage for cross-fitted residuals
Y_res = np.zeros_like(Y)
D_res = np.zeros_like(D)

# ----- SIMPLE SHALLOW NN -----
nn = MLPRegressor(
    hidden_layer_sizes=(10,),   # 1 hidden layer, 10 neurons
    activation='tanh',          # smooth + stable
    # solver='lbfgs',             # deterministic, best for small nets
    solver='adam',
    max_iter=500,
    random_state=42
)

for train_idx, test_idx in kf.split(X):

    # --- ML: predict E[Y | X] ---
    nn.fit(X[train_idx], Y[train_idx])
    Y_res[test_idx] = Y[test_idx] - nn.predict(X[test_idx])

    # --- ML: predict E[D | X] ---
    nn.fit(X[train_idx], D[train_idx])
    D_res[test_idx] = D[test_idx] - nn.predict(X[test_idx])


# =========================================================
# 4. DML-IV: LINEAR IV ON RESIDUALs
# =========================================================

# FIRST STAGE: D_res ~ Z
X_first = add_constant(Z)
first_stage = OLS(D_res, X_first).fit()
D_hat = first_stage.fittedvalues
print(first_stage.summary())

# SECOND STAGE: Y_res ~ D_hat
X_second = add_constant(D_hat)
second_stage = OLS(Y_res, X_second).fit()
# print(second_stage.summary())

# =========================================================
# 5. Clustered SE
# =========================================================

clusters = df[["muni_code","cl_microYear"]].copy()
clusters["muni_code"] = clusters["muni_code"].astype(int)
clusters["cl_microYear"] = clusters["cl_microYear"].astype("category").cat.codes.astype(int)

ss_cluster = second_stage.get_robustcov_results(
    cov_type="cluster",
    groups=clusters
)

print("\n========== DML-IV (Shallow NN) with Clustered SE ==========")
print(ss_cluster.summary())


                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                    0.1916
Date:                Mon, 01 Dec 2025   Prob (F-statistic):              0.662
Time:                        20:42:35   Log-Likelihood:                -22403.
No. Observations:                5210   AIC:                         4.481e+04
Df Residuals:                    5208   BIC:                         4.482e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0643      0.247      0.260      0.7

In [137]:
import numpy as np
from scipy.stats import norm

# from second-stage OLS on residuals
theta_hat = second_stage.params[1]
theta_hat

# ------------------------------------------------------------
# 0. Required objects
# ------------------------------------------------------------

# cluster IDs as 1D numpy arrays
cluster1 = np.asarray(clusters["muni_code"].values)
cluster2 = np.asarray(clusters["cl_microYear"].values)

# orthogonal scores: ψ_i = Z_res * (Y_res - theta_hat * D_res)
psi = Z * (Y_res - theta_hat * D_res)

# Jacobian:  J = -(1/n) * Σ Z_res * D_res
J = -np.mean(Z * D_res)

n = len(psi)

# ------------------------------------------------------------
# 1. Compute cluster sums of ψ_i
# ------------------------------------------------------------
def cluster_sum(psi_vec, cluster_id_vec):
    cluster_id_vec = np.asarray(cluster_id_vec)
    out = {}
    for cid in np.unique(cluster_id_vec):
        mask = (cluster_id_vec == cid)
        out[cid] = psi_vec[mask].sum()
    return out

# cluster 1 sums
C1 = cluster_sum(psi, cluster1)
# cluster 2 sums
C2 = cluster_sum(psi, cluster2)

# intersection clusters: create a single combined cluster label
cluster12 = np.array([f"{a}_{b}" for a, b in zip(cluster1, cluster2)])
C12 = cluster_sum(psi, cluster12)

# ------------------------------------------------------------
# 2. Variance components
# ------------------------------------------------------------
V1  = sum(v**2 for v in C1.values())
V2  = sum(v**2 for v in C2.values())
V12 = sum(v**2 for v in C12.values())

# Two-way cluster variance (Cameron–Gelbach–Miller)
cluster_var = (V1 + V2 - V12) / (n**2)

# ------------------------------------------------------------
# 3. Delta method for θ (Jacobian scaling)
# ------------------------------------------------------------
Var_theta = cluster_var / (J**2)
se_dml_tw = np.sqrt(Var_theta)

# ------------------------------------------------------------
# 4. CI
# ------------------------------------------------------------
z_95 = norm.ppf(0.975)
ci_low = theta_hat - z_95 * se_dml_tw
ci_high = theta_hat + z_95 * se_dml_tw

print(f"Theta_hat (DML-IV): {theta_hat:.6f}")
print(f"Two-Way Clustered DML SE: {se_dml_tw:.6f}")
print(f"95% CI (two-way cluster): ({ci_low:.6f}, {ci_high:.6f})")


Theta_hat (DML-IV): -0.436789
Two-Way Clustered DML SE: 1.041206
95% CI (two-way cluster): (-2.477515, 1.603936)


### 5. Partially Linear DML-IV (XGB)

In [131]:
from xgboost import XGBRegressor

# =========================================================
# 3. PARTIALLY LINEAR DML (ML TO PARTIAL OUT CONTROLS)
# =========================================================

# variables (demeaned)
Y = df_w[y_var].values           # deforest
D = df_w[endog].values           # fines
Z = df_w[instr].values           # cloud instrument
X = X_controls.values            # nonlinear controls

K = 5
kf = KFold(n_splits=K, shuffle=True, random_state=42)

# storage for cross-fitted residuals
Y_res = np.zeros_like(Y)
D_res = np.zeros_like(D)

# =========================================================
# XGBoost Model (stable for DML)
# =========================================================
xgb = XGBRegressor(
    n_estimators=400,
    max_depth=3,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    min_child_weight=10,
    objective='reg:squarederror',
    reg_alpha=1.0,       # L1 penalty
    reg_lambda=1.0,      # L2 penalty
    random_state=42
)

for train_idx, test_idx in kf.split(X):

    # --- ML: predict E[Y | X] ---
    xgb.fit(X[train_idx], Y[train_idx])
    Y_res[test_idx] = Y[test_idx] - xgb.predict(X[test_idx])

    # --- ML: predict E[D | X] ---
    xgb.fit(X[train_idx], D[train_idx])
    D_res[test_idx] = D[test_idx] - xgb.predict(X[test_idx])


# =========================================================
# 4. DML-IV: LINEAR IV ON RESIDUALs
# =========================================================

# FIRST STAGE: D_res ~ Z
X_first = add_constant(Z)
first_stage = OLS(D_res, X_first).fit()
D_hat = first_stage.fittedvalues
print(first_stage.summary())

# SECOND STAGE: Y_res ~ D_hat
X_second = add_constant(D_hat)
second_stage = OLS(Y_res, X_second).fit()


# =========================================================
# 5. Clustered SE
# =========================================================

clusters = df[["muni_code","cl_microYear"]].copy()
clusters["muni_code"] = clusters["muni_code"].astype(int)
clusters["cl_microYear"] = clusters["cl_microYear"].astype("category").cat.codes.astype(int)

ss_cluster = second_stage.get_robustcov_results(
    cov_type="cluster",
    groups=clusters
)

print("\n========== DML-IV (Shallow NN) with Clustered SE ==========")
print(ss_cluster.summary())


                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     2.282
Date:                Mon, 01 Dec 2025   Prob (F-statistic):              0.131
Time:                        16:50:42   Log-Likelihood:                -22481.
No. Observations:                5210   AIC:                         4.497e+04
Df Residuals:                    5208   BIC:                         4.498e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0342      0.251     -0.137      0.8

In [132]:
import numpy as np
from scipy.stats import norm

# from second-stage OLS on residuals
theta_hat = second_stage.params[1]
theta_hat

# ------------------------------------------------------------
# 0. Required objects
# ------------------------------------------------------------

# cluster IDs as 1D numpy arrays
cluster1 = np.asarray(clusters["muni_code"].values)
cluster2 = np.asarray(clusters["cl_microYear"].values)

# orthogonal scores: ψ_i = Z_res * (Y_res - theta_hat * D_res)
psi = Z * (Y_res - theta_hat * D_res)

# Jacobian:  J = -(1/n) * Σ Z_res * D_res
J = -np.mean(Z * D_res)

n = len(psi)

# ------------------------------------------------------------
# 1. Compute cluster sums of ψ_i
# ------------------------------------------------------------
def cluster_sum(psi_vec, cluster_id_vec):
    cluster_id_vec = np.asarray(cluster_id_vec)
    out = {}
    for cid in np.unique(cluster_id_vec):
        mask = (cluster_id_vec == cid)
        out[cid] = psi_vec[mask].sum()
    return out

# cluster 1 sums
C1 = cluster_sum(psi, cluster1)
# cluster 2 sums
C2 = cluster_sum(psi, cluster2)

# intersection clusters: create a single combined cluster label
cluster12 = np.array([f"{a}_{b}" for a, b in zip(cluster1, cluster2)])
C12 = cluster_sum(psi, cluster12)

# ------------------------------------------------------------
# 2. Variance components
# ------------------------------------------------------------
V1  = sum(v**2 for v in C1.values())
V2  = sum(v**2 for v in C2.values())
V12 = sum(v**2 for v in C12.values())

# Two-way cluster variance (Cameron–Gelbach–Miller)
cluster_var = (V1 + V2 - V12) / (n**2)

# ------------------------------------------------------------
# 3. Delta method for θ (Jacobian scaling)
# ------------------------------------------------------------
Var_theta = cluster_var / (J**2)
se_dml_tw = np.sqrt(Var_theta)

# ------------------------------------------------------------
# 4. CI
# ------------------------------------------------------------
z_95 = norm.ppf(0.975)
ci_low = theta_hat - z_95 * se_dml_tw
ci_high = theta_hat + z_95 * se_dml_tw

print(f"Theta_hat (DML-IV): {theta_hat:.6f}")
print(f"Two-Way Clustered DML SE: {se_dml_tw:.6f}")
print(f"95% CI (two-way cluster): ({ci_low:.6f}, {ci_high:.6f})")


Theta_hat (DML-IV): -0.057451
Two-Way Clustered DML SE: 0.062184
95% CI (two-way cluster): (-0.179329, 0.064427)


### 6. Flexible DML-IV (RF)

In [ ]:
# =========================================================
# FLEXIBLE PARTIALLY LINEAR IV (RF version)
# =========================================================

Y = df_w[y_var].values
D = df_w[endog].values
Z = df_w[instr].values.reshape(-1,1)   # must be 2D for concatenation
X = X_controls.values

XZ = np.hstack([Z, X])   # for gamma^(3)

K = 5
kf = KFold(n_splits=K, shuffle=True, random_state=42)

Y_res = np.zeros_like(Y)
D_res = np.zeros_like(D)
instr_opt = np.zeros_like(D)

rf = RandomForestRegressor(
    n_estimators=500,
    max_depth=None,
    min_samples_leaf=5,
    random_state=42
)

for train_idx, test_idx in kf.split(X):

    # gamma^(1): E[Y|X]
    rf.fit(X[train_idx], Y[train_idx])
    Y_res[test_idx] = Y[test_idx] - rf.predict(X[test_idx])

    # gamma^(2): E[D|X]
    rf.fit(X[train_idx], D[train_idx])
    D_res[test_idx] = D[test_idx] - rf.predict(X[test_idx])

    # gamma^(3): E[D|Z,X]  <-- flexible IV
    rf.fit(XZ[train_idx], D[train_idx])
    instr_opt[test_idx] = rf.predict(XZ[test_idx])
    

# FIRST STAGE: D_res ~ instr_opt
X_first = add_constant(instr_opt)
first_stage = OLS(D_res, X_first).fit()
D_hat = first_stage.fittedvalues
print(first_stage.summary())

# SECOND STAGE: Y_res ~ D_hat
X_second = add_constant(D_hat)
second_stage = OLS(Y_res, X_second).fit()

print(second_stage.summary())


                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                    0.6546
Date:                Mon, 01 Dec 2025   Prob (F-statistic):              0.419
Time:                        16:29:16   Log-Likelihood:                -4851.0
No. Observations:                5210   AIC:                             9706.
Df Residuals:                    5208   BIC:                             9719.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0052      0.009     -0.612      0.5

In [123]:
print(first_stage.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.010
Model:                            OLS   Adj. R-squared:                  0.010
Method:                 Least Squares   F-statistic:                     53.43
Date:                Mon, 01 Dec 2025   Prob (F-statistic):           3.08e-13
Time:                        16:43:06   Log-Likelihood:                -22367.
No. Observations:                5210   AIC:                         4.474e+04
Df Residuals:                    5208   BIC:                         4.475e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0326      0.245     -0.133      0.8

### 7. Partially DML-IV (RF) - without weather variables in first stage

In [142]:
# ---------------------------------------------------------
# Basic variables (demeaned)
# ---------------------------------------------------------
Y = df_w[y_var].values
D = df_w[endog].values
Z = df_w[instr].values.reshape(-1,1)
X_full = X_controls.values
X_cols = X_controls.columns.tolist()

restricted_cols = [
    c for c in X_cols
    if ("weather" not in c.lower())
    and ("temp" not in c.lower())
    and ("rain" not in c.lower())
]

X_restricted = X_controls[restricted_cols].values

K = 5
kf = KFold(n_splits=K, shuffle=True, random_state=42)

Y_res = np.zeros_like(Y)
D_res = np.zeros_like(D)

rf = RandomForestRegressor(
    n_estimators=500,
    max_depth=None,
    min_samples_leaf=5,
    random_state=42
)

for train_idx, test_idx in kf.split(X_full):

    # gamma^(1): E[Y|X_full]
    rf.fit(X_full[train_idx], Y[train_idx])
    Y_res[test_idx] = Y[test_idx] - rf.predict(X_full[test_idx])

    # gamma^(2): E[D|X_restricted]
    rf.fit(X_restricted[train_idx], D[train_idx])
    D_res[test_idx] = D[test_idx] - rf.predict(X_restricted[test_idx])

X_first = add_constant(Z)
first_stage = OLS(D_res, X_first).fit()
D_hat = first_stage.fittedvalues

print("\n===== PL-RF FIRST STAGE (restricted X) =====")
print(first_stage.summary())

X_second = add_constant(D_hat)
second_stage = OLS(Y_res, X_second).fit()

print("\n===== PL-RF SECOND STAGE (restricted X) =====")
print(second_stage.summary())




===== PL-RF FIRST STAGE (restricted X) =====
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     2.767
Date:                Mon, 01 Dec 2025   Prob (F-statistic):             0.0963
Time:                        22:49:26   Log-Likelihood:                -22393.
No. Observations:                5210   AIC:                         4.479e+04
Df Residuals:                    5208   BIC:                         4.480e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const 

In [143]:
import numpy as np
from scipy.stats import norm

# ------------------------------------------------------------
# INPUTS YOU ALREADY HAVE:
#  - theta_hat      (second-stage coefficient)
#  - Y_res, D_res   (partialled-out residuals)
#  - Z              (the raw instrument, NOT residualized)
# ------------------------------------------------------------

# Orthogonal score: ψ_i = Z_i * (Y_res_i - θ * D_res_i)
psi = Z * (Y_res - theta_hat * D_res)

# Jacobian:  J = -(1/n) * Σ Z_i * D_res_i
J = -np.mean(Z * D_res)

n = len(psi)

# ------------------------------------------------------------
# Variance of ψ_i
# ------------------------------------------------------------
Var_psi = np.mean((psi - psi.mean())**2)

# ------------------------------------------------------------
# Delta-method variance for θ
# Var(θ) = Var(ψ_i) / (n * J^2)
# ------------------------------------------------------------
Var_theta = Var_psi / (n * (J**2))
se_theta = np.sqrt(Var_theta)

# ------------------------------------------------------------
# CI
# ------------------------------------------------------------
z_95 = norm.ppf(0.975)
ci_low = theta_hat - z_95 * se_theta
ci_high = theta_hat + z_95 * se_theta

print("Theta_hat:", theta_hat)
print("Jacobian J:", J)
print("DML Standard Error (no clustering):", se_theta)
print("95% CI:", (ci_low, ci_high))


Theta_hat: -0.037599444636163395
Jacobian J: 7.329473661415485e-21
DML Standard Error (no clustering): 2.222582217788821e+17
95% CI: (-4.356181099545248e+17, 4.356181099545248e+17)


In [ ]:
import numpy as np
from scipy.stats import norm

# from second-stage OLS on residuals
theta_hat = second_stage.params[1]
theta_hat

# ------------------------------------------------------------
# 0. Required objects
# ------------------------------------------------------------

# cluster IDs as 1D numpy arrays
cluster1 = np.asarray(clusters["muni_code"].values)
cluster2 = np.asarray(clusters["cl_microYear"].values)

# orthogonal scores: ψ_i = Z_res * (Y_res - theta_hat * D_res)
psi = Z * (Y_res - theta_hat * D_res)

# Jacobian:  J = -(1/n) * Σ Z_res * D_res
J = -np.mean(Z * D_res)

n = len(psi)

# ------------------------------------------------------------
# 1. Compute cluster sums of ψ_i
# ------------------------------------------------------------
def cluster_sum(psi_vec, cluster_id_vec):
    cluster_id_vec = np.asarray(cluster_id_vec)
    out = {}
    for cid in np.unique(cluster_id_vec):
        mask = (cluster_id_vec == cid)
        out[cid] = psi_vec[mask].sum()
    return out

# cluster 1 sums
C1 = cluster_sum(psi, cluster1)
# cluster 2 sums
C2 = cluster_sum(psi, cluster2)

# intersection clusters: create a single combined cluster label
cluster12 = np.array([f"{a}_{b}" for a, b in zip(cluster1, cluster2)])
C12 = cluster_sum(psi, cluster12)

# ------------------------------------------------------------
# 2. Variance components
# ------------------------------------------------------------
V1  = sum(v**2 for v in C1.values())
V2  = sum(v**2 for v in C2.values())
V12 = sum(v**2 for v in C12.values())

# Two-way cluster variance (Cameron–Gelbach–Miller)
cluster_var = (V1 + V2 - V12) / (n**2)

# ------------------------------------------------------------
# 3. Delta method for θ (Jacobian scaling)
# ------------------------------------------------------------
Var_theta = cluster_var / (J**2)
se_dml_tw = np.sqrt(Var_theta)

# ------------------------------------------------------------
# 4. CI
# ------------------------------------------------------------
z_95 = norm.ppf(0.975)
ci_low = theta_hat - z_95 * se_dml_tw
ci_high = theta_hat + z_95 * se_dml_tw

print(f"Theta_hat (DML-IV): {theta_hat:.6f}")
print(f"Two-Way Clustered DML SE: {se_dml_tw:.6f}")
print(f"95% CI (two-way cluster): ({ci_low:.6f}, {ci_high:.6f})")


Theta_hat (DML-IV): -0.037599
Two-Way Clustered DML SE: 25867468582248288256.000000
95% CI (two-way cluster): (-50699306792428019712.000000, 50699306792428019712.000000)
